In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import torch
from timesfm import TimesFm, TimesFmHparams, TimesFmCheckpoint
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

In [ ]:
# Parameters and Settings

# Parameters for data split
WINDOW = 21    # rolling window size to use as predictors
DATE_COL = 'Date'
ID_COL = 'PERMNO'
TARGET_COL = 'excess_return'

# File path for the cleaned and filtered data file
current_directory = os.getcwd()
clean_filtered_data_path = os.path.join(current_directory, 'Data', 'clean_filtered_data.csv')

# File path to save prediction results
results_path = os.path.join(current_directory, 'Results', f'timesfm_models_results{WINDOW:.0f}.csv')

# Estimation (in sample) period dates
in_sample_start_date = pd.to_datetime("2000-01-01")
in_sample_end_date = pd.to_datetime("2015-12-31")

# Out-of-sample period dates
out_sample_start_date = pd.to_datetime("2016-01-01")
out_sample_end_date = pd.to_datetime("2024-12-31")

# Use GPU if available, else default to using CPU
device_map = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Step 1: Load and Process Data

In [ ]:
# Load the cleaned and filtered data files for in sample and out of sample periods into a pandas DataFrames
df = pd.read_csv(clean_filtered_data_path)

# Ensure the date columns are in datetime format
df[DATE_COL] = pd.to_datetime(df[DATE_COL])

df = df[[ID_COL, DATE_COL, TARGET_COL]].dropna()
df = df.sort_values([ID_COL, DATE_COL]).reset_index(drop=True)
df.info()

In [ ]:
# Check number of unique stocks
stocks_permno = df["PERMNO"].unique().tolist()
print(f"Number of unique stocks: {len(stocks_permno)}")

In [ ]:
# Spit data into estimation (in-sample) and out-of-sample data
df_train = df[(df[DATE_COL] >= in_sample_start_date) & (df[DATE_COL] <= in_sample_end_date)].copy().reset_index(drop=True)
out_sample_start_date = df_train[DATE_COL].tail(WINDOW).iloc[0]
df_test = df[(df[DATE_COL] >= out_sample_start_date) & (df[DATE_COL] <= out_sample_end_date)].copy().reset_index(drop=True)

print(df_train.info())
print(df_test.info())

In [ ]:
# Create rolling window for predictors
contexts = []
targets = []
records = []

for id, grp in df_test.groupby(ID_COL):
    values = grp[TARGET_COL].values
    dates = grp[DATE_COL].values
    for i in range(len(values) - WINDOW):
        contexts.append(values[i:i+WINDOW])
        targets.append(values[i+WINDOW])
        records.append({
            ID_COL: id,
            TARGET_COL: values[i+WINDOW],
            DATE_COL: dates[i+WINDOW]
        })

In [ ]:
y_test = pd.Series(targets)

results = pd.DataFrame(records)

### Step 2: Zero-Shot Forecasting with TimesFM

In [ ]:
# Creating a Function to Calculate Predictive-R2 Used in the Finance Literature
def r2(y_true, y_pred):
    return 1-(((y_true-y_pred)**2).sum()/(y_true**2).sum())

In [ ]:
# Zero Shot TimesFM-1.0-200M
tfm1 = TimesFm(
    hparams = TimesFmHparams(
        context_len = int(32 * np.ceil(WINDOW / 32)),   # context length should be multiple of 32
        horizon_len = 1,
        input_patch_len = 32,                           # fixed for 200m model
        output_patch_len = 128,                         # fixed for 200m model
        num_layers = 20,                                # fixed for 200m model
        model_dims = 1280,                              # fixed for 200m model
        backend = device_map       
        ),
    checkpoint = TimesFmCheckpoint(huggingface_repo_id="google/timesfm-1.0-200m-pytorch")
    )

freqs = [0] * len(contexts)
preds, _ = tfm1.forecast(contexts, freq=freqs)

y_tfm1 = pd.Series(preds.reshape([-1,]))

results['y_tfm1'] = y_tfm1

In [ ]:
# Zero Shot TimesFM-2.0-500M
tfm2 = TimesFm(
    hparams = TimesFmHparams(
        context_len = int(32 * np.ceil(WINDOW / 32)),   # context length should be multiple of 32
        horizon_len = 1,
        input_patch_len = 32,                           # fixed for 500m model
        output_patch_len = 128,                         # fixed for 500m model
        num_layers = 50,                                # fixed for 500m model
        model_dims = 1280,                              # fixed for 500m model
        backend = device_map       
        ),
    checkpoint = TimesFmCheckpoint(huggingface_repo_id="google/timesfm-2.0-500m-pytorch")
    )

freqs = [0] * len(contexts)
preds, _ = tfm2.forecast(contexts, freq=freqs)

y_tfm2 = pd.Series(preds.reshape([-1,]))

results['y_tfm2'] = y_tfm2

### Step 3: Evaluate Statistical Performance of Models

In [ ]:
# Evaluate models

# TimesFM 1.0
r2_tfm1  = r2(y_test, y_tfm1)
mse_tfm1 = mean_squared_error(y_test, y_tfm1)
mae_tfm1 = mean_absolute_error(y_test, y_tfm1)
da_tfm1 = (np.sign(y_test) == np.sign(y_tfm1)).mean()

# TimesFM 2.0
r2_tfm2  = r2(y_test, y_tfm2)
mse_tfm2 = mean_squared_error(y_test, y_tfm2)
mae_tfm2 = mean_absolute_error(y_test, y_tfm2)
da_tfm2 = (np.sign(y_test) == np.sign(y_tfm2)).mean()

In [ ]:
# Collating Results

results_matrix = [{
        "Model": "TimesFM-1.0-200M",
        "R-squared": r2_tfm1,
        "MSE": mse_tfm1,
        "MAE": mae_tfm1,
        "Direction Accuracy": da_tfm1
    },
    {
        "Model": "TimesFM-2.0-500M",
        "R-squared": r2_tfm2,
        "MSE": mse_tfm2,
        "MAE": mae_tfm2,
        "Direction Accuracy": da_tfm2
    }]

results_matrix_df = pd.DataFrame(results_matrix)
results_matrix_df

##### Save Results

In [ ]:
# Save Prediction Results
results.to_csv(results_path, index=False)